In [ ]:
import csv
import pickle
import os
import re
import functools
import collections
import sys
if os.path.exists('C:/Users/Tianle/Documents/cs231n/spring1617/my-scripts'):
    sys.path.append('C:/Users/Tianle/Documents/cs231n/spring1617/my-scripts')
    root = 'C:/Users/Tianle/Documents/cs231n/spring1617/my-scripts/'
if os.path.exists('/projects/academic/azhang/tianlema/deeplearning'):
    sys.path.append('/projects/academic/azhang/tianlema/deeplearning')
    root = '/projects/academic/azhang/tianlema/deeplearning/'
sys.path.append('/projects/academic/azhang/tianlema/deeplearning/site-packages/')    
import numpy as np
import matplotlib.pyplot as plt

from dl.utils.gen_conv_params import *

%load_ext autoreload
%autoreload 2

# Read clinical data

In [ ]:
file='../notebooks/globalClinTraining.csv'
clinical = []
with open(file) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        clinical.append(row)
# The first row is feature names
var_names = clinical[0]
# Map feature name to idx
var_names = {name: i for i, name in enumerate(var_names)}
del clinical[0]

In [ ]:
# Patient ID are unique
assert len([v[1] for v in clinical]) == len(set([v[1] for v in clinical]))

# SamplId are unique if not 'NA'
# This allows using (filename, sampleID) or just sampleID to find patientID
for name in [v for v in var_names if re.search('SamplId', v)]:
    sids = [v[var_names[name]] for v in clinical
            if v[var_names[name]] != 'NA']
    print(name, len(sids))
    assert len(sids) == len(set(sids))

# Find expression files in two folders: MA and RNA-seq
expfiles = [(root, files) for root, subdir, files in os.walk('Expression Data/') if len(files) > 0]
# Select files with entrezID
expfilepaths = [[os.path.join(root, f) for f in files if re.search('entrezID', f)] 
                for root, files in expfiles]
# Save filenames
expfilenames = [[f for f in files if re.search('entrezID', f)] for root, files in expfiles]

In [ ]:
import vcf
def get_mutation_info(filename):
    reader = vcf.Reader(filename=filename)
    pos = []
    ann = []
    for r in reader:
        info = r.INFO['ANN'][0].split('|')
        pos.append({'CHROM': r.CHROM, 'POS': r.POS, 'REF': r.REF, 'ALT': info[0]})
        ann.append(info)
    return pos, ann

In [ ]:
path = ('/projects/academic/jamesjar/tianlema/dl-datasets/mm-dream/data/Genomic Data/'
        'MMRF IA9 CelgeneProcessed/filtered Strelka SnpSift Annotated vcfs/snvs/')
snv_files = [(i, path+v[var_names['WES_mutationFileStrelkaSNV']]) for i, v in enumerate(clinical)
             if v[var_names['WES_mutationFileStrelkaSNV']] != 'NA']

In [ ]:
snv = []
clinical_snv = []
for i, filename in snv_files:
    try:
        snv.append(get_mutation_info(filename))
        clinical_snv.append(clinical[i])
    except ValueError:
        print(i, filename)

In [ ]:
with open('snv.pkl', 'wb') as f:
    pickle.dump({'snv': snv, 'clinical': clinical_snv}, f)

# Select a subset of mutations

In [ ]:
with open('snv.pkl', 'rb') as f:
    data = pickle.load(f)

snv = data['snv']
clinical = data['clinical']

genes = [set([v[3] for v in r[1]]) for r in snv]
genes = functools.reduce(lambda a, b: a.union(b), genes)

# # Use biomart to convert gene symbols to entrezIds # use Bioconductor
# with open('genesymbols.csv', 'w') as f:
#     for g in genes:
#         f.write(g+'\n')
sym2entrez = []
with open('../processed_data/sym2entrez.csv', 'r') as f:
    reader = csv.reader(f)
    for r in reader:
        sym2entrez.append(r)
sym_to_entrez = collections.defaultdict(list)
for k, v in sym2entrez:
    sym_to_entrez[k].append(v)

In [ ]:
mutations = [{(s[3], s[1], s[2]) for s in v[1]} for v in snv]
mut_occ = collections.defaultdict(list)
for i, m in enumerate(mutations):
    for k in m:
        mut_occ[k].append(i)
# plt.hist([len(v) for k, v in mut_occ.items()], bins=[s for s in range(0, 100, 10)]+[1000])

# select genes that has a mapped entrez ID (this may not be necessary)
# and are mutated in at least 5% of the samples or impact is 'HIGH'
tmp = collections.defaultdict(set)
for k,v in mut_occ.items():
    tmp[k[0]].update(v)
mut_occ = {k:v for k, v in mut_occ.items() if len(tmp[k[0]])>36 or k[2]=='HIGH'}
mut_occ = {k:v for k, v in mut_occ.items() if k[0] in sym_to_entrez}

In [ ]:
# order mutations by their positions for possible future use
mut_pos = collections.defaultdict(set)
for r in snv:
    for s, t in zip(r[0], r[1]):
        mut_pos[(t[3], t[1], t[2])].add('{0}.{1}'.format(s['CHROM'], s['POS']))
tmp = {k: sorted(v)[0] for k, v in mut_pos.items() if k in mut_occ}
pos = {v:i for i, v in enumerate(sorted(tmp.values()))}
mut_ids = {k: pos[v] for k, v in tmp.items()}

In [ ]:
with open('snv.pkl', 'rb') as f:
    data = pickle.load(f)
    

data['mut_ids'] = mut_ids
with open('snv-input.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
# with open('snv-input.pkl', 'wb') as f:
#     data_snv = pickle.load(f)

# mut_ids = data_snv['mut_ids']
# cli_idx = {(v[0], v[1]):i for i,v in enumerate(clinical)}

# snv_mat = np.zeros((len(snv),len(mut_ids)))

# for i, m in enumerate(mutations):
#     idx = [v for k, v in mut_ids.items() if k in m]
#     snv_mat[i, idx] = 1

# np.sum(snv_mat)